In [ ]:
from sklearn.mixture import GaussianMixture
import scipy.stats as stats
from scipy.stats import multivariate_normal
import scipy as sp
from sklearn.cluster import KMeans

from sklearn.decomposition import PCA
import sklearn.mixture
import sklearn
from scipy import sparse
import matplotlib
import scipy
import pickle
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(context='talk', style='white', palette='colorblind')

from python_tsne_utils import utils
# import cpm matrices 
import importlib

In [ ]:
black_background_style = {
    'axes.facecolor': 'black',
    'axes.edgecolor': 'white',
    'axes.labelcolor': 'white',
    'figure.facecolor': 'black',
    'figure.edgecolor': 'black',
    'xtick.color': 'white',
    'ytick.color': 'white',
    'grid.color': 'gray',
    'text.color': 'white',
    'lines.color': 'white',
    'patch.edgecolor': 'white',
    'savefig.facecolor': 'black',
    'savefig.edgecolor': 'black',
    'legend.facecolor': 'black',
    'legend.edgecolor': 'white',
    'legend.fontsize': 'large',
    'legend.title_fontsize': 'large',
    'font.family': 'monospace',
    'font.monospace': 'Courier New',
}

# Apply the custom style
plt.rcParams.update(black_background_style)

In [ ]:
def lognormalize_counts(tasic_dict):
    
    counts = tasic_dict['counts']
    
    # trying to catch all formats in which the counts might be loaded
    if scipy.sparse.issparse(counts):
        counts = counts.toarray()
    elif isinstance(CPM, np.matrix):
        counts = np.squeeze(np.asarray(counts))
    else:
        raise TypeError(f"Data format is {type(counts)} but should be np.martix or a sparse matrix.")
    
    #normalize and logtransform counts
    libsizes = counts.sum(axis=1)
    CPM = counts / libsizes[:, None] * 1e+6
        
    logCPM = np.log2(CPM + 1) 
    tasic_dict['logCPM'] = logCPM  
    
    return tasic_dict

In [ ]:
tasic_1k = lognormalize_counts(pickle.load(open('tasic_subset_1kselected.pickle', 'rb'))) 

In [ ]:
print(tasic_1k['logCPM'].shape)

In [ ]:
# choose a subset for test
np.random.seed(42)
subset = np.random.choice(tasic_1k['logCPM'].shape[0], 1000, replace=False)
data = tasic_1k['logCPM'][subset, :]
clusters = tasic_1k['clusters'][subset]
labels = tasic_1k['clusterColors'][clusters]
print(data.shape)

In [ ]:
tsne_results = utils.tsne_with_dof_optimisation(X=data, n_iter = 500, initial_alpha=1, dataset_name='tasic_1k_subset')

In [ ]:
utils.plot_tsne_result(tsne_results, labels=labels)

In [ ]:
tasic_1k = lognormalize_counts(pickle.load(open('tasic_subset_1kselected.pickle', 'rb'))) 
data = tasic_1k['logCPM']
clusters = tasic_1k['clusters']
labels = tasic_1k['clusterColors'][clusters]
print(data.shape)
print(labels.shape)

In [ ]:
tsne_results = utils.tsne_with_dof_optimisation(X=data, n_iter = 500, initial_alpha=1, dataset_name='tasic_1k')

In [ ]:
import importlib
importlib.reload(utils) 
utils.plot_tsne_result(tsne_results, labels=labels)

In [ ]:
tsne_results_std = utils.run_reference_tsne(X=data, n_iter = 500, fixed_alpha=1, dataset_name='tasic_1k')

In [ ]:
utils.plot_tsne_result(tsne_results_std, labels=labels)

In [ ]:
importlib.reload(utils)
tsne_result_opt_1 = utils.tsne_with_dof_optimisation(
    X=data,
    n_iter=500,
    initial_alpha=1,
    dataset_name="tasic_1k",
    optimise_for_alpha="exact",
    alpha_lr=0.5,
    verbose=True,
    num_threads=16,
    
)

In [ ]:
utils.plot_tsne_result(tsne_result_opt_1, labels, additional_title='Exact optimised')

In [ ]:
importlib.reload(utils)
utils.plot_side_by_side(tsne_results, tsne_result_opt_1, labels, additional_title_2="Exact optimised")

In [ ]:
tsne_result_opt_bh = utils.tsne_with_dof_optimisation(X=data, n_iter=500, initial_alpha=1, dataset_name="tasic_1k", optimise_for_alpha="bh", alpha_lr=0.5, verbose=True, num_threads=16)

In [ ]:

importlib.reload(utils)
knn_recall = utils.compute_knn_recall(data, tsne_result_opt_bh.embedding, labels, k=10)
from python_tsne_utils.utils import TSNEResultsWithKNN
tsne_result_opt_bh_with_knn = TSNEResultsWithKNN(**vars(tsne_result_opt_bh), knn_recall=knn_recall)

In [ ]:
utils.plot_tsne_result(tsne_result_opt_bh, labels, additional_title="BH-optimised")

In [ ]:
utils.plot_side_by_side(tsne_result_opt_1, tsne_result_opt_bh, labels, additional_title_2="BH-optimised", additional_title_1="Exact optimised")

## Full tasic


In [ ]:
tasic_data = np.load('Tasic raw data/tasic-pca50.npy') # pca with 50 components
clusters  = np.load('Tasic raw data/tasic-ttypes.npy')
labels = np.load('Tasic raw data/tasic-colors.npy')

In [ ]:
print(tasic_data.shape)
print(clusters.shape)
print(labels.shape)

In [ ]:
tsne_result_full_tasic_bh = utils.tsne_with_dof_optimisation(X=tasic_data, n_iter=500, initial_alpha=1, dataset_name="Tasic", optimise_for_alpha="bh", alpha_lr=0.5, verbose=True, num_threads=16)

In [ ]:
importlib.reload(utils)
knn_recall = utils.compute_knn_recall(original_data=tasic_data, tsne_data=tsne_result_full_tasic_bh.embedding, k=10)
from python_tsne_utils.utils import TSNEResultsWithKNN
tsne_result_opt_bh_with_knn = TSNEResultsWithKNN(**vars(tsne_result_full_tasic_bh), knn_recall=knn_recall)

In [ ]:
utils.save_pickle(tsne_result_opt_bh_with_knn, 'tsne_result_full_tasic_bh_with_knn.pickle')

In [ ]:
utils.plot_tsne_result(tsne_result_opt_bh_with_knn, labels, additional_title="BH-optimised")

In [ ]:
tsne_result_full_tasic_no_opt = utils.tsne_with_dof_optimisation(X=tasic_data, n_iter=500, initial_alpha=1, dataset_name="Tasic", verbose=True, num_threads=16)

In [ ]:
utils.plot_tsne_result(tsne_result_full_tasic_no_opt, labels, additional_title="No optimisation")

In [ ]:
knn_recall = utils.compute_knn_recall(original_data=tasic_data, tsne_data=tsne_result_full_tasic_no_opt.embedding, k=10)
from python_tsne_utils.utils import TSNEResultsWithKNN
tsne_result_full_tasic_no_opt_with_knn = TSNEResultsWithKNN(**vars(tsne_result_full_tasic_no_opt), knn_recall=knn_recall)

In [ ]:
tsne_result_opt_bh_with_knn = utils.load_pickle('Processed data/tsne_result_full_tasic_bh_with_knn.pickle')

In [ ]:
importlib.reload(utils)
utils.plot_side_by_side_plotly(tsne_result_full_tasic_no_opt_with_knn, tsne_result_opt_bh_with_knn, labels, additional_title_2="Optimized with Barnes-Hutt", additional_title_1="No optimization")

In [ ]:
import pickle
with open('full_tasic_tsne_results.pickle', 'wb') as f:
    pickle.dump(tsne_result_full_tasic_bh, f)

In [ ]:
from openTSNE import TSNE

In [ ]:
regular_tsne_auto_params = TSNE(n_iter=500, random_state=42, n_jobs=16, verbose=True)
regular_tsne_results = regular_tsne_auto_params.fit(tasic_data)

In [ ]:
plt.scatter (regular_tsne_results[:, 0], regular_tsne_results[:, 1], c=labels, cmap='tab20', s = 0.3)

In [ ]:
tsne_result_opt_bh_with_knn = utils.load_pickle('Processed data/tsne_result_full_tasic_bh_with_knn.pickle')

In [ ]:
utils.plot_tsne_result(tsne_result_opt_bh_with_knn, labels, additional_title="BH-optimised")

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import numpy as np
from IPython.display import HTML


plt.rcParams['animation.embed_limit'] = 1000  # Increase the limit to 1000 MB (1 GB)
# Example data: list of t-SNE embeddings
tsne_embeddings = tsne_result_opt_bh_with_knn.im_embeddings
#labels 
# Create a figure and axis
fig, ax = plt.subplots()
sc = ax.scatter(tsne_embeddings[0][:, 0], tsne_embeddings[0][:, 1], c=labels, s=0.3)
# ax.set_xlim(-100, 100)
# ax.set_ylim(-100, 100)
# Remove axes, labels, and spines
ax.set_xticks([])
ax.set_yticks([])
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['bottom'].set_visible(False)
ax.spines['left'].set_visible(False)
def init():
    sc.set_offsets(np.c_[tsne_embeddings[0][:, 0], tsne_embeddings[0][:, 1]])
    return sc,

def update(i):
    sc.set_offsets(np.c_[tsne_embeddings[i][:, 0], tsne_embeddings[i][:, 1]])
    ax.set_xlim(np.min(tsne_embeddings[i][:, 0]), np.max(tsne_embeddings[i][:, 0]))
    ax.set_ylim(np.min(tsne_embeddings[i][:, 1]), np.max(tsne_embeddings[i][:, 1]))
    return sc,

# Create the animation
ani = animation.FuncAnimation(fig, update, frames=len(tsne_embeddings), init_func=init, blit=True)

# Save the animation as a GIF file using Pillow
ani.save('Figures/tsne_mnist_full_opt_bh.gif', writer='imagemagick', fps=10, dpi = 300)


# Display the animation in the Jupyter Notebook
HTML(ani.to_jshtml())